In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pathlib import Path
# Coneccion base de datos
from sqlalchemy import create_engine
import cx_Oracle

In [14]:
# Conectar con base de datos
import configparser

config = configparser.ConfigParser()

config.read('../config.ini')

# Obtener los datos de conexión a la base de datos
host = config['oracle']['host']
port = config['oracle']['port']
sid = config['oracle']['service_name']
user = config['oracle']['user']
password = config['oracle']['password']

# Configuración de la base de datos
dsn = cx_Oracle.makedsn(host, port, sid)

engine = create_engine(f'oracle+cx_oracle://{user}:{password}@{dsn}')

In [7]:
import requests
from bs4 import BeautifulSoup
import os

# URL de la página web
url = "https://www.minsalud.gov.co/salud/POS/Paginas/unidad-de-pago-por-capitacion-upc.aspx"

# Obtener el contenido de la página web con verificación SSL deshabilitada
response = requests.get(url, verify=False)
html_content = response.content

# Analizar el contenido HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Encontrar todos los enlaces
links = soup.find_all('a')

# Directorio para guardar los archivos descargados
download_dir = '../downloads'
os.makedirs(download_dir, exist_ok=True)

# Descargar los archivos ZIP
for link in links:
    href = link.get('href')
    if href and href.endswith('Vigencia%202024.zip'):
        file_url = href if href.startswith('http') else f"https://www.minsalud.gov.co{href}"  # Ajusta si es necesario
        file_name = os.path.join(download_dir, os.path.basename(href))
        print(f"Descargando {file_url} a {file_name}")
        zip_response = requests.get(file_url, verify=False)
        with open(file_name, 'wb') as file:
            file.write(zip_response.content)

print("Descarga completada.")


c:\Users\camil\anaconda3\envs\actualizacion_parametricas_upc_py3.11\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.minsalud.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Descargando https://www.minsalud.gov.co/sites/rid/Lists/BibliotecaDigital/RIDE/VP/RBC/Vigencia%202024.zip a ../downloads\Vigencia%202024.zip


c:\Users\camil\anaconda3\envs\actualizacion_parametricas_upc_py3.11\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.minsalud.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Descarga completada.


In [8]:
# Abrir zip y extraer archivos
import zipfile

def extract_zip(zip_file, extract_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)


extract_zip('../downloads/Vigencia%202024.zip', '../extracted_files')        

In [2]:
ruta = Path(r"D:\Keralty scripts\Automatizaciones-python\actualizacion_parametricas_upc\extracted_files\Vigencia 2024\2024_TR_CIE10_05_IMG_WEB_26Mar2024.xlsb")
df = pd.read_excel(ruta, skiprows=4, engine='pyxlsb')

In [3]:
df.head()

,VIGENCIA,CODIGO,DESCRIPCION,SEXO,EDAD_LIM_INF,EDAD_LIM_SUP
0,2024,A000,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO CH...",A,0,120
1,2024,A001,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO EL...",A,0,120
2,2024,A009,"COLERA, NO ESPECIFICADO",A,0,120
3,2024,A010,FIEBRE TIFOIDEA,A,0,120
4,2024,A011,FIEBRE PARATIFOIDEA A,A,0,120


In [5]:
df.shape

(12634, 6)

In [8]:
# Revisar si hay unicos
df.loc[df.duplicated(subset='CODIGO')]

,VIGENCIA,CODIGO,DESCRIPCION,SEXO,EDAD_LIM_INF,EDAD_LIM_SUP


## Estructura tabla TBL_SUF_CIE10_2024

| Columna                                | Tipo         |
|----------------------------------------|--------------|
| Tabla                                  | VARCHAR2(255)|
| CAPÍTULO                              | VARCHAR2(255)|
| NOMBRE CAPÍTULO               | VARCHAR2(255)|
| CIE10                                  | VARCHAR2(255)|
| DESCRIPCIÓN CÓDIGOS DE CUATRO CARACTERES | VARCHAR2(255)|
| CODIGO_ CIE-10_ TRES_CARACTERES | VARCHAR2(255)|
| DESCRIPCION_CODIGOS_TRES_CARACTERES | VARCHAR2(255)|
| SEXO                                   | VARCHAR2(255)|
| LIMITE_INFERIOR                 | VARCHAR2(255)|
| LIMITE_SUPERIOR                | VARCHAR2(255)|
| GRUPO_ MORTALIDAD_LISTA_ 6-607 | VARCHAR2(255)|
| GRUPO_MORTALIDAD_LISTA_105 | VARCHAR2(255)|
| VALOR_LIM_INF                      | VARCHAR2(255)|
| VALOR_LIM_SUP                     | VARCHAR2(255)|
| TIPO_LIMI_INFERIOR              | VARCHAR2(20)|
| TIPO_LIMI_SUPERIOR             | VARCHAR2(20)|

In [10]:

df.rename(columns={'VIGENCIA': 'Tabla', 'CODIGO':'CIE10', 'DESCRIPCION': 'DESCRIPCIÓN CÓDIGOS DE CUATRO CARACTERES', 'EDAD_LIM_INF': 'VALOR_LIM_INF','EDAD_LIM_SUP': 'VALOR_LIM_SUP'}, inplace=True)

In [12]:
df.columns.to_list()

['Tabla',
 'CIE10',
 'DESCRIPCIÓN CÓDIGOS DE CUATRO CARACTERES',
 'SEXO',
 'VALOR_LIM_INF',
 'VALOR_LIM_SUP']

In [15]:
# Copiar estructura de la tabla EPSS05LIQUIDACION_202405 para crear la tabla EPSS05LIQUIDACION_202406

with engine.connect() as connection:
    
    # Crear cursor
    cursor = connection.connection.cursor()
    
        # Borra tabla si existe
    try:
        cursor.execute(f"""
            TRUNCATE TABLE TBL_SUF_CIE10_2024
        """)    
    except Exception as e:
        print(e)
    

# Hacer la insercion por chunk df_liquidados
df.to_sql('tbl_suf_cie10_2024', con=engine, if_exists='append', index=False)

12634